In [1]:
!pip install imbalanced-learn


In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# 한글 폰트 설정 (Windows)
mpl.rc('font', family='Malgun Gothic')

# 마이너스 기호 깨짐 방지
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
import pandas as pd
import numpy as np
import mariadb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, r2_score
from imblearn.over_sampling import SMOTE  # 데이터 불균형 해결

# 📌 1. MariaDB 연결 설정
DB_CONFIG = {
    "user": "root",
    "password": "1234",
    "host": "localhost",
    "database": "finfit"
}

def load_data_from_db():
    """MariaDB에서 데이터를 불러오는 함수"""
    conn = mariadb.connect(**DB_CONFIG)
    query = "SELECT * FROM medical"
    df = pd.read_sql(query, conn)
    conn.close()
    return df.dropna()

# 📌 2. 데이터 불러오기
health_data = load_data_from_db()
print(f"✅ DB에서 데이터 로드 완료! 총 {len(health_data)}개의 데이터\n")

# 📌 3. 사용할 컬럼 정의 (정답 데이터는 X에서 제거!)
columns_to_use = [
    'sex', 'age', 'height', 'weight', 'bmi', 'alchol', 'smoking_history',
    'chol_total', 'chol_hdl', 'chol_ldl', 'chol_tg',
    'fasting_blood_sugar', 'glycated_hemoglobin',
    'sbp_average', 'dbp_average', 'cancer_diagnosis_fathers',
    'cancer_diagnosis_mother', 'cancer_diagnosis_sibling',
    'white_blood_cell_count', 'red_blood_cell_count', 'stress'
]

# 📌 4. 암 위험도 계산 함수
def calculate_cancer_risk(row):
    return (
        (row['cancer_diagnosis_fathers'] * 3) +  
        (row['cancer_diagnosis_mother'] * 3) +  
        (row['cancer_diagnosis_sibling'] * 3) +  
        (row['chol_total'] / 30) +  
        (row['sbp_average'] / 15) +  
        (row['dbp_average'] / 15) +  
        (row['fasting_blood_sugar'] / 15) +  
        (row['glycated_hemoglobin'] * 2.5) +  
        (row['white_blood_cell_count'] * 1.5) +  
        (row['red_blood_cell_count'] * 1.2)
    )

# 암 위험도 점수 추가
health_data['cancer_risk_score'] = health_data.apply(calculate_cancer_risk, axis=1)

# 📌 5. 정답 데이터(타겟 변수) 정의
disease_labels = {
    '고지혈증': 'dyslipidemia_status',
    '당뇨': 'diabetes',
    '고혈압': 'high_blood_pressure'
}

# 📌 6. 입력 데이터(X)와 정답 데이터(y) 분리
X = health_data[columns_to_use]  # ✅ 정답 데이터 제외
y_disease = health_data[list(disease_labels.values())]  # ✅ 질병 예측용 정답 데이터
y_cancer = health_data["cancer_risk_score"]  # ✅ 암 위험도 예측용 정답 데이터

# 📌 7. 결측값 처리 (비어 있는 데이터가 있어도 학습 가능하도록)
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="mean")  # 평균값으로 결측값 채우기
X_imputed = imputer.fit_transform(X)

# 📌 8. 정규화 (StandardScaler 사용)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# 📌 9. 데이터 분할 (Train/Test) → `X_test`를 공통으로 사용
X_train, X_test, y_train_disease, y_test_disease, y_train_cancer, y_test_cancer = train_test_split(
    X_scaled, y_disease, y_cancer, test_size=0.2, random_state=42
)

# 📌 10. SMOTE 적용 (데이터 불균형 해결) → 질병 예측에만 적용
# 📌 10. SMOTE 적용 (각 질병별로 개별 적용)
smote = SMOTE(random_state=42)

X_train_smote_dict = {}  # SMOTE 적용된 X_train 저장
y_train_smote_dict = {}  # SMOTE 적용된 y_train 저장

for disease, label in disease_labels.items():
    X_train_smote_dict[disease], y_train_smote_dict[disease] = smote.fit_resample(X_train, y_train_disease[label])


# 📌 11. 질병 예측 모델 학습 (SMOTE 적용된 데이터셋 사용)
optimized_models = {}
best_params = {}
predicted_probs = {}

param_grid = {
    'n_estimators': [200],
    'max_depth': [15],
    'min_samples_split': [5],
    'max_features': ['sqrt']
}

for disease, label in disease_labels.items():
    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
    grid_search.fit(X_train_smote_dict[disease], y_train_smote_dict[disease])  # ✅ SMOTE 적용된 데이터 사용
    best_params[disease] = grid_search.best_params_

    model = RandomForestClassifier(**best_params[disease], random_state=42)
    model.fit(X_train_smote_dict[disease], y_train_smote_dict[disease])  # ✅ SMOTE 적용된 데이터 사용
    predicted_probs[disease] = model.predict_proba(X_test)[:, 1]
    optimized_models[disease] = model


# 📌 12. 암 위험도 예측 모델 학습 (RandomForestRegressor 사용)
cancer_model = RandomForestRegressor(n_estimators=200, random_state=42)
cancer_model.fit(X_train, y_train_cancer)
predicted_probs["암"] = cancer_model.predict(X_test)

# 📌 13. 암 위험도 점수 백분위 변환
health_data["cancer_risk_percentile"] = health_data["cancer_risk_score"].rank(pct=True, ascending=True) * 100
cancer_risk_percentile = health_data.loc[y_test_cancer.index, "cancer_risk_percentile"].values

# 📌 14. 위험도 분류 함수 추가
def classify_general_risk(prob):
    """일반적인 유병 확률을 저위험/중위험/고위험으로 변환"""
    if prob < 0.10:
        return "저위험"
    elif prob < 0.30:
        return "중위험"
    else:
        return "고위험"

def classify_cancer_risk(percentile):
    """암 위험도 순위를 바탕으로 위험도 분류"""
    if percentile <= 10:
        return "고위험"
    elif percentile <= 30:
        return "중위험"
    else:
        return "저위험"

# 📌 15. 최종 결과 정리 (위험도 추가!)
risk_data = pd.DataFrame({
    "고지혈증 유병 확률": predicted_probs["고지혈증"],
    "당뇨 유병 확률": predicted_probs["당뇨"],
    "고혈압 유병 확률": predicted_probs["고혈압"],
    "암 위험도 점수": predicted_probs["암"],
    "암 위험도": [classify_cancer_risk(x) for x in cancer_risk_percentile],
    "암 위험도 순위": [f"상위 {int(x)}%" for x in cancer_risk_percentile],  # ✅ 백분위 변환
})

# 📌 **위험도(저위험/중위험/고위험) 정보 추가**
for disease in ["고지혈증", "당뇨", "고혈압"]:
    risk_data[f"{disease} 위험도"] = risk_data[f"{disease} 유병 확률"].apply(classify_general_risk)  # ✅ 위험도 추가!

# 📌 확률 값을 %로 변환하는 함수
def format_percentage(value):
    return f"{value * 100:.1f}%"  # 예: 0.231 → 23.1%

# 📌 고지혈증, 당뇨, 고혈압 유병 확률을 %로 변환
for disease in ["고지혈증", "당뇨", "고혈압"]:
    risk_data[f"{disease} 유병 확률"] = risk_data[f"{disease} 유병 확률"].apply(format_percentage)

# ✅ 최종 결과 출력
print("\n📊 유병 확률 및 위험도 분석 결과")
display(risk_data.head(30))


C:\Users\arsey\AppData\Local\Temp\ipykernel_412\416432983.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ DB에서 데이터 로드 완료! 총 4599개의 데이터



c:\venvs\finfit\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\venvs\finfit\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\arsey\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 550, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\arsey\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1028, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\arsey\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1540, in _execut


📊 유병 확률 및 위험도 분석 결과


,고지혈증 유병 확률,당뇨 유병 확률,고혈압 유병 확률,암 위험도 점수,암 위험도,암 위험도 순위,고지혈증 위험도,당뇨 위험도,고혈압 위험도
0,3.3%,1.1%,0.0%,50.584542,고위험,상위 7%,저위험,저위험,저위험
1,0.8%,0.0%,0.0%,53.500517,저위험,상위 35%,저위험,저위험,저위험
2,100.0%,0.0%,2.2%,58.657195,저위험,상위 70%,고위험,저위험,저위험
3,1.0%,0.6%,0.1%,50.473817,중위험,상위 10%,저위험,저위험,저위험
4,1.6%,0.2%,37.4%,53.828863,저위험,상위 34%,저위험,저위험,고위험
5,1.7%,0.1%,0.4%,50.961570,중위험,상위 10%,저위험,저위험,저위험
6,4.5%,0.8%,0.0%,55.498062,저위험,상위 64%,저위험,저위험,저위험
7,2.2%,0.0%,0.9%,53.832978,저위험,상위 39%,저위험,저위험,저위험
8,1.1%,6.0%,1.2%,53.885707,저위험,상위 34%,저위험,저위험,저위험
9,93.3%,1.2%,3.6%,58.208812,저위험,상위 66%,고위험,저위험,저위험


In [4]:
import pickle
import os

# ✅ 모델 저장 경로 설정
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)  # models 폴더 없으면 생성

# ✅ 모델 저장 (customer_features 제거)
model_path = os.path.join(model_dir, "disease_model.pkl")
with open(model_path, "wb") as f:
    pickle.dump({
        "optimized_models": optimized_models,
        "cancer_model": cancer_model,
        "scaler": scaler,
        "imputer": imputer
    }, f)

print(f"✅ 모델 저장 완료: {model_path}")


✅ 모델 저장 완료: models\disease_model.pkl
